In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [238]:
response = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=J6ZBYR30XCBEQYKW')
response = response.json()['Time Series (5min)']

In [239]:
atr_period = 7
multiplier = 3

In [240]:
with open('data.json', 'w') as file:
    json.dump(response, file)

In [241]:
df = pd.read_json('data.json').transpose()
df.reset_index(level=0, inplace=True)
df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Volume'] = df['Volume'].astype(np.int64)

In [242]:
df = df[::-1]
df.reset_index(level=0, inplace=True)
df = df.drop(['index'], axis=1)

In [243]:
daily = df.copy()

def atr(DF,n):
    df=DF.copy()
    df['High-Low']=abs(df['High']-df['Low'])
    df['High-PreviousClose']=abs(df['High']-df['Close'].shift(1))
    df['Low-PreviousClose']=abs(df['Low']-df["Close"].shift(1))
    df['TrueRange']=df[['High-Low','High-PreviousClose','Low-PreviousClose']].max(axis=1,skipna=False)
    df['ATR']=df['TrueRange'].ewm(com=n,min_periods=n).mean()
    return df['ATR']
n=10
daily['ATR']=atr(daily,n)


def supertrend(DF,n,m):
    df = DF.copy()
    df['ATR'] = atr(df,n)
    df["BASIC_UPPERBAND"]=((df['High']+df['Low'])/2) + m*df['ATR'] 
    df["BASIC_LOWERBAND"]=((df['High']+df['Low'])/2) - m*df['ATR']
    df["FINAL_UPPERBAND"]=df["BASIC_UPPERBAND"]
    df["FINAL_LOWERBAND"]=df["BASIC_LOWERBAND"]
    ind = df.index
    for i in range(n,len(df)):
        if df['Close'][i-1]<=df['FINAL_UPPERBAND'][i-1]:
            df.loc[ind[i],'FINAL_UPPERBAND']=min(df['BASIC_UPPERBAND'][i],df['FINAL_UPPERBAND'][i-1])
        else:
            df.loc[ind[i],'FINAL_UPPERBAND']=df['BASIC_UPPERBAND'][i]    
    for i in range(n,len(df)):
        if df['Close'][i-1]>=df['FINAL_LOWERBAND'][i-1]:
            df.loc[ind[i],'FINAL_LOWERBAND']=max(df['BASIC_LOWERBAND'][i],df['FINAL_LOWERBAND'][i-1])
        else:
            df.loc[ind[i],'FINAL_LOWERBAND']=df['BASIC_LOWERBAND'][i]  
    df['Strend']=np.nan
    for test in range(n,len(df)):
        if df['Close'][test-1]<=df['FINAL_UPPERBAND'][test-1] and df['Close'][test]>df['FINAL_UPPERBAND'][test]:
            df.loc[ind[test],'Strend']=df['FINAL_LOWERBAND'][test]
            break
        if df['Close'][test-1]>=df['FINAL_LOWERBAND'][test-1] and df['Close'][test]<df['FINAL_LOWERBAND'][test]:
            df.loc[ind[test],'Strend']=df['FINAL_UPPERBAND'][test]
            break
    for i in range(test+1,len(df)):
        if df['Strend'][i-1]==df['FINAL_UPPERBAND'][i-1] and df['Close'][i]<=df['FINAL_UPPERBAND'][i]:
            df.loc[ind[i],'Strend']=df['FINAL_UPPERBAND'][i]
        elif  df['Strend'][i-1]==df['FINAL_UPPERBAND'][i-1] and df['Close'][i]>=df['FINAL_UPPERBAND'][i]:
            df.loc[ind[i],'Strend']=df['FINAL_LOWERBAND'][i]
        elif df['Strend'][i-1]==df['FINAL_LOWERBAND'][i-1] and df['Close'][i]>=df['FINAL_LOWERBAND'][i]:
            df.loc[ind[i],'Strend']=df['FINAL_LOWERBAND'][i]
        elif df['Strend'][i-1]==df['FINAL_LOWERBAND'][i-1] and df['Close'][i]<=df['FINAL_LOWERBAND'][i]:
            df.loc[ind[i],'Strend']=df['FINAL_UPPERBAND'][i]
    return df['Strend']


def val_to_bool(val, close):
    if val > close:
        return True
    elif val < close:
        return False

daily['Supertrend']=supertrend(df,atr_period,multiplier)
print(daily.to_string())

             Timestamp      Open      High       Low     Close  Volume       ATR  Supertrend
0  2022-12-19 16:15:00  138.8700  138.8700  138.8100  138.8100   13133       NaN         NaN
1  2022-12-19 16:20:00  138.8700  138.8700  138.8100  138.8100   12428       NaN         NaN
2  2022-12-19 16:40:00  138.7500  138.7500  138.7500  138.7500     529       NaN         NaN
3  2022-12-19 16:45:00  138.7500  138.7500  138.6002  138.6002     488       NaN         NaN
4  2022-12-19 17:10:00  138.6300  138.6300  138.5900  138.5900    3092       NaN         NaN
5  2022-12-19 17:20:00  138.5500  138.5500  138.5500  138.5500     300       NaN         NaN
6  2022-12-19 18:05:00  138.7900  138.8100  138.7900  138.8100     695       NaN         NaN
7  2022-12-19 18:20:00  138.8400  138.8400  138.7200  138.7200     935       NaN         NaN
8  2022-12-19 18:30:00  138.5600  138.5600  138.5600  138.5600     232       NaN         NaN
9  2022-12-19 19:10:00  138.8800  138.8800  138.8800  138.8800     105

In [244]:
def superTrendIndicator(df, atr_period, multiplier):
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    price_difference = [high - low, 
                        high - close.shift(), 
                        close.shift() - low]
    true_range = pd.concat(price_difference, axis=1)
    true_range = true_range.abs().max(axis=1)
    atr = true_range.ewm(alpha=1/atr_period, min_periods=atr_period).mean()
    
    hl2 = (high + low) /  2
    
    final_upper_band = upper_band = hl2 + (multiplier * atr)
    final_lower_band = lower_band = hl2 - (multiplier * atr)
    
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        if close[curr] > final_upper_band[prev]:
            supertrend[curr] = True
        elif close[curr] < final_lower_band[prev]:
            supertrend[curr] = False
        else:
            supertrend[curr] = supertrend[prev]
            
            if supertrend[curr] is True and final_lower_band[curr] < final_lower_band[prev]:
                
                final_lower_band[curr] = final_lower_band[prev]
                
            if supertrend[curr] is False and final_upper_band[curr] > final_upper_band[prev]:
                
                final_upper_band[curr] = final_upper_band[prev]
            
        
        if supertrend[curr] is True:
            final_upper_band[curr] = np.nan
        else:
            final_lower_band[curr] = np.nan
        
        return pd.DataFrame({
            'Supertrend': supertrend,
            'Lower Band': final_lower_band,
            'Upper Band': final_upper_band
        }, index=df.index)

In [245]:
s = superTrendIndicator(df, atr_period, multiplier)
df = df.join(s)
df

,Timestamp,Open,High,Low,Close,Volume,Supertrend,Lower Band,Upper Band
0,2022-12-19 16:15:00,138.87,138.87,138.8100,138.8100,13133,True,NaN,NaN
1,2022-12-19 16:20:00,138.87,138.87,138.8100,138.8100,12428,True,NaN,NaN
2,2022-12-19 16:40:00,138.75,138.75,138.7500,138.7500,529,True,NaN,NaN
3,2022-12-19 16:45:00,138.75,138.75,138.6002,138.6002,488,True,NaN,NaN
4,2022-12-19 17:10:00,138.63,138.63,138.5900,138.5900,3092,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,2022-12-20 16:20:00,141.25,141.25,141.2500,141.2500,100,True,140.854099,141.645901
96,2022-12-20 16:55:00,141.28,141.28,141.2800,141.2800,2200,True,140.927799,141.632201
97,2022-12-20 17:05:00,141.12,141.12,141.1200,141.1200,1174,True,140.749542,141.490458
98,2022-12-20 19:50:00,141.40,141.40,141.4000,141.4000,100,True,140.962465,141.837535
